# 100% RE by 2050 and the Effects of Lifetime and Recycling

Comparison case using the functions in PV ICE to compare how PV module lifetime and PV module recycling effect the energy transition to 100% clean energy by 2050.





![Folder 15 vs 50 year Module](../images_wiki/2_15vs50Overview.PNG)

WORDS WORDS WORDS


In [1]:
import PV_ICE
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
from IPython.display import display
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

import os
from pathlib import Path

PV_ICE.__version__

'v0.2.0+61.g5f5316e.dirty'

In [21]:
testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP' / 'SFs_LifeVSRecycle')
inputfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP')
print ("Your simulation will be stored in %s" % testfolder)
print("Your simulation input data will be written/read from %s" % inputfolder)

Your simulation will be stored in C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\TEMP\SFs_LifeVSRecycle
Your simulation input data will be written/read from C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\TEMP


In [3]:
if not os.path.exists(testfolder):
    os.makedirs(testfolder)

# File Preparation

In [16]:
reedsFile = str(Path().resolve().parent.parent.parent / 'December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx')
print ("Input file is stored in %s" % reedsFile)

Input file is stored in C:\Users\hmirletz\Documents\GitHub\December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx


In [18]:
REEDSInput = pd.read_excel(reedsFile, sheet_name="new installs PV")
#NEED TO CHECK THAT AC to DC has been taken care of in excel file (there is a multiplier, is set to 1)

In [19]:
rawdf = REEDSInput.copy()
rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True) #tech=pvtotal from "new installs PV sheet", so can drop
rawdf.set_index(['Scenario','Year','PCA'], inplace=True)

Loading Module Baseline. Will be used later to populate all the columns other than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model. Unlike the SF simulations, this analysis will use PV ICE developed baselines.

In [23]:
rtest = PV_ICE.Simulation(name='Sim1', path=inputfolder)
rtest.createScenario(name='test', file=r'..\baselines\baseline_modules_US.csv') #likely to cause year issue
baseline = rtest.scenario['test'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head(5)

path = C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repair,mod_MerchantTail,mod_Reuse
year,,,,,,,,,,,
1995,12.50,16.0,21.0,0.55,10.0,98.0,15.0,40.0,0.0,0.0,0.0
1996,12.70,16.0,21.0,0.55,10.0,98.0,15.0,40.0,0.0,0.0,0.0
1997,12.88,19.0,23.0,0.55,15.0,98.0,15.0,40.0,0.0,0.0,0.0
1998,13.06,19.0,23.0,0.55,15.0,98.0,15.0,40.0,0.0,0.0,0.0
1999,13.24,19.0,23.0,0.55,15.0,98.0,15.0,40.0,0.0,0.0,0.0


Collect all the scenario names and downselect to the scenario(s) of interest. In this case, we are only concerned with the highest capacity and deployment rate, Decarbonization + Electrification (Decarb+E)

In [4]:
scenarios = ['Reference.Mod',
 'Reference.Adv',
 'Reference.Adv_DR',
 '95-by-35.Mod',
 '95-by-35.Adv',
 '95-by-35.Adv_DR',
 '95-by-35_Elec.Mod',
 '95-by-35_Elec.Adv',
 '95-by-35_Elec.Adv_DR']

SFscenarios = ['95-by-35_Elec.Adv_DR'] #Decarb+E
SFscenarios

['95-by-35_Elec.Adv_DR']

In [7]:
#add materials for simulation
MATERIALS = ['glass','aluminium_frames','silicon','silver', 'copper', 'encapsulant', 'backsheet']

Set up the PV ICE simulation with scenario and materials

In [10]:
MATERIALBASELINE = r'..\..\baselines\baseline_material_'+MATERIALS[0]+'.csv'
MATERIALBASELINE

'..\\..\\baselines\\baseline_material_glass.csv'

In [14]:
modulefile = SFscenarios[0]+'.csv'
os.path.join(inputfolder, 'USA', modulefile) #need USA bc that's where ReEDS Scenarios Baseline Creation stores its output)

'C:\\Users\\hmirletz\\Documents\\GitHub\\PV_ICE\\PV_ICE\\TEMP\\USA\\95-by-35_Elec.Adv_DR.csv'

In [ ]:
r1 = PV_ICE.Simulation(name='SF-LvR', path=testfolder)

for scen in range(len(SFscenarios)):
    modulefile = SFscenarios[scen]+'.csv'
    modulefile = os.path.join(inputfolder, 'USA', modulefile)
    r1.createScenario(name='Decarb+E_PVICE_defaults', file=modulefile) #change name=SFscenarios[scen] if multiple scenarios
    for mat in range(len(MATERIALS)):
        MATERIALBASELINE = r'..\..\baselines\baseline_material_'+MATERIALS[mat]+'.csv'
        r1.scenario[SFscenarios[scen]].addMaterial(MATERIALS[mat], file=MATERIALBASELINE)